# Evaluation <a name = "Top"></a>

# Quick Links

<ol>
    <li><a href = #setup>Setup</a></li>
    <li><a href = #plots>Plots</a></li>
</ol>

## Imports

In [ ]:
import os
import shutil

In [ ]:
from datetime import datetime, timedelta
import time
import pandas as pd
import numpy as np
import shutil

from enum import Enum

from numpy import array

import ntpath

import copy
import re

## Hilfsfunktionen

### Enum für Trainingsset

In [ ]:
class TrainingSet(Enum):
    SYNTHETIC = 1
    REAL = 2
    MIXED = 3

### Enum für Label-Typ

In [ ]:
class LabelType(Enum):
    ANGULAR = 1
    STEREOGRAPHIC = 2

### Trainingsset-Typ nach String Converter

In [ ]:
def trainingset_to_string(ts):
    if ts == TrainingSet.SYNTHETIC:
        return 'Synth'
    elif ts == TrainingSet.REAL:
        return 'Real'
    elif ts == TrainingSet.MIXED:
        return 'Mixed'
    else:
        print('Unknown TrainingSet')
        return None

### LabelType nach String

In [ ]:
def labeltype_to_string(lt):
    if lt == LabelType.ANGULAR:
        return 'Angular'
    if lt == LabelType.STEREOGRAPHIC:
        return 'Stereographic'
    else:
        print('Unknown LabelType')
        return None

#### Required format of parameters parameter for _model_predict_ (...)

In [ ]:
p = {
    'dataset_to_use':[],
    'model_to_load':[],
    'dataset_name':'combined_dataset',
    'activation_function':[],
    'leaky_ReLU_alpha':[],
    'first_neuron':[],
    'dropout_rate':[],
    'hidden_layers':[],
    'optimizer':[],
    'learning_rate':[],
    'loss_function':[],
    'label_type':[]
}

## Hilfsfunktionen

### Konvertierung ($S_x$, $S_y$) $\rightarrow$ ($\phi$, $\theta$)

In [ ]:
def convert_from_stereographic(sx, sy, r = 1):
    
    l = np.sqrt(sx * sx + sy * sy)
    theta = 90 - 2 * np.degrees(np.arctan(l / 2))
    
    if sx < 0:
        phi = 180 - np.degrees(np.arcsin(sy / l))
        
    elif sx >= 0:
        if sy > 0:
            phi = np.degrees(np.arcsin(sy / l))
        elif sy < 0:
            phi = 360 + np.degrees(np.arcsin(sy / l))
        else:
            #phi1 = np.NaN
            phi = 0
    else:
        print('sx and sy undefined. should not have reached here')

    return phi, theta

### Konvertierung ($\phi$, $\theta$) $\rightarrow$ ($S_x$, $S_y$)

In [ ]:
def convert_from_spheric(phi, theta, r = 1):
    m = 2 * r * np.tan(np.radians((90 - theta) / 2))
    sy = m * np.sin(np.radians(phi))
    sx = m * np.cos(np.radians(phi))
    return sx, sy

### Radians $\rightarrow$ Degree

In [ ]:
def to_degree(angle_in_rad):
    return angle_in_rad * 180 / np.pi

### Degree $\rightarrow$ Radians

In [ ]:
def to_radians(angle_in_deg):
    return angle_in_deg * np.pi / 180

### Sphärische $\rightarrow$ Karthesische Koordinaten

In [ ]:
def spheric_cartesian_polar(phi_d, theta_d):
    x = np.sin(np.radians(90.0 - theta_d)) * np.cos(np.radians(phi_d))
    y = np.sin(np.radians(90.0 - theta_d)) * np.sin(np.radians(phi_d))
    z = np.cos(np.radians(90.0 - theta_d))
    return array([x, y, z])

In [ ]:
def spheric_cartesian_elevation(phi_d, theta_d):
    x = np.cos(np.radians(theta_d)) * np.cos(np.radians(phi_d))
    y = np.cos(np.radians(theta_d)) * np.sin(np.radians(phi_d))
    z = np.sin(np.radians(theta_d))
    return array([x, y, z])

### Length of Vector

In [ ]:
def vectorlength(vector):
    return np.linalg.norm(vector)

### Calculated Angular Error

In [ ]:
def calculate_angular_error(deg_e_phi, deg_e_theta):
    return np.degrees(np.arccos(np.cos(np.radians(deg_e_phi)) * np.cos(np.radians(deg_e_theta))))

### Skalarprodukt

In [ ]:
def myDot(a, b):
    dot = 0;
    it = np.nditer(a, flags=['f_index'])
    for x in it:
        dot = dot + (x * b[it.index])
        
    return dot

### Dot Angular Error

In [ ]:
def dot_angular_error_elevation(predicted_deg_vector, true_deg_vector):    
    c_predicted = spheric_cartesian_elevation(predicted_deg_vector[0], predicted_deg_vector[1])
    c_true = spheric_cartesian_elevation(true_deg_vector[0], true_deg_vector[1])
    
    len_prediction = vectorlength(c_predicted)
    len_true = vectorlength(c_true)
    
    cos_angle = np.dot(c_true, c_predicted) / len_prediction / len_true
    
    return abs(np.degrees(np.arccos(cos_angle)))

def dot_angular_error_polar(predicted_deg_vector, true_deg_vector):
    c_predicted = spheric_cartesian_polar(predicted_deg_vector[0], predicted_deg_vector[1])
    c_true = spheric_cartesian_polar(true_deg_vector[0], true_deg_vector[1])
    
    len_prediction = vectorlength(c_predicted)
    len_true = vectorlength(c_true)
    
    cos_angle = np.dot(c_true, c_predicted) / len_prediction / len_true
    
    return abs(np.degrees(np.arccos(cos_angle)))

## Normierte sphärische Koordinaten

### Normierte $\rightarrow$ Sphärische

In [ ]:
def normalized_to_spheric(n_phi, n_theta):
    phi = n_phi * 180 + 180
    theta = n_theta * 45 + 45
    
    return phi, theta

# Evaluation <a name = "setup"></a>
<p><a href = #Top>Up</a>
<p><a href = #plots>Plots</a>

In [ ]:
run = 'SYNTH'
loss = 'MSE'
dataset_name = '2020-05-28'
net_index = 195
APPENDIX = 'Stereographic'
_note = '_Custom-MAE'

eval_dir = '..\\output\\{}_Regression_{}\\Graphical_Evaluation\\'.format(run, loss)

if(not os.path.exists(eval_dir)):
    os.makedirs(eval_dir)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(eval_dir))

In [ ]:
import pickle

with open(eval_dir + '{}_Net{}_{}{}_Results.pickle'.format(dataset_name, net_index, APPENDIX, _note), "rb") as fp:   # Unpickling
    mse_results = pickle.load(fp)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.text as text
from matplotlib.pyplot import figure

import tikzplotlib
%matplotlib inline

Enable_Plotting = True

# Plots <a name = "plots">
<p></a><a href = #Top>Up</a>
<p><a href = #setup>Setup</a>

In [ ]:
if Enable_Plotting:
    avg_error_list = []

    for i in range(len(mse_results)):
        net_id = re.findall(r'Idx\d+', mse_results[i][0])
        net_id = re.findall(r'\d+', net_id[0])
        net_trained = re.findall(r'[A-Z,a-z]*_TD', mse_results[i][0])
        df_index = '{}-{}-Net-{}\n Train {} Test {}'.format(dataset_name, labeltype_to_string(mse_results[i][4]), net_id[0], net_trained[0], trainingset_to_string(mse_results[i][2]))

        cp_df = copy.deepcopy(mse_results[i][3])
        cp_df = cp_df.drop(['Avg_Elevation_Err', 'Avg_Azimuth_Err', 'Avg_Sx_Err', 'Avg_Sy_Err', 'Avg_Inference_Time'], axis = 1)
        cp_df.insert(0, 'Net', [df_index], True)
        cp_df.insert(1, 'Test Data', [mse_results[i][2]], True)
        cp_df.set_index('Net')

        avg_error_list.append(cp_df)

In [ ]:
if Enable_Plotting:
    list_df = pd.concat(avg_error_list)
list_df

##  Plot Variance Diagram

In [ ]:
def do_Plot(indexlist, title, eval_file, save = True):
    fig, ax = plt.subplots(figsize = (7.5,7.5))
    #ax = plt.gca()
    
    df_list = []
    for index in indexlist:
        df_list.append(avg_error_list[index])
    
    plot_df = pd.concat(df_list)
    
    x = plot_df['Net']
    y = plot_df['Avg_Angular_Err']
    e = np.sqrt(plot_df['Angular_Variance'])

    plot_df = plot_df[['Net', 'Test Data', 'Avg_Angular_Err', 'Angular_Variance']]

    (_, caps, _) = plt.errorbar(x, y, e, linestyle = 'None', marker = 'o', markersize = 3, figure = fig, capsize = 3)
    
    for cap in caps:
        cap.set_markeredgewidth(1)
    
    plt.setp(ax.get_xticklabels(), rotation = 'vertical')
    plt.ylabel('Average Angular Error\n & \nVariance')
    #plt.legend(['Average Angular Error'])
    plt.title(title)
    ax.grid(True, ls = '--', axis = 'y')
    
    for o in fig.findobj(text.Text):
        o.set_fontstyle('italic')
    
    if(save):
        tikzplotlib.save('{}.tex'.format(eval_file))
        plt.savefig('{}.png'.format(eval_file), format = 'png', bbox_inches = "tight", dpi = 300)

    plt.show()
    
    return plot_df

### Train: S, Test: S

In [ ]:
if Enable_Plotting:
    tsts_plot = do_Plot([0, 6, 12, 18, 24], 'Train: S, Test: S', eval_dir + 'Error-Variance_Train_S_Test_S')

## Train: R, Test: S

In [ ]:
if Enable_Plotting:
    trts_plot = do_Plot([1, 7, 13, 19, 25], 'Train: R, Test: S', eval_dir + 'Error-Variance_Train_R_Test_S')

## Train: M, Test: S

In [ ]:
if Enable_Plotting:
    tmts_plot = do_Plot([2, 8, 14, 20, 26], 'Train: M, Test: S', eval_dir + 'Error-Variance_Train_M_Test_S')

## Train: S, Test: R

In [ ]:
if Enable_Plotting:
    tstr_plot = do_Plot([3, 9, 15, 21, 27], 'Train: S, Test: R', eval_dir + 'Error-Variance_Train_S_Test_R')

## Train: R, Test: R

In [ ]:
if Enable_Plotting:
    trtr_plot = do_Plot([4, 10, 16, 22, 28], 'Train: R, Test: R', eval_dir + 'Error-Variance_Train_R_Test_R')

## Train: M, Test: R

In [ ]:
if Enable_Plotting:
    tmtr_plot = do_Plot([5, 11, 17, 23, 29], 'Train: M, Test: R', eval_dir + 'Error-Variance_Train_M_Test_R')